<a href="https://colab.research.google.com/github/Loptt/CE888-Course/blob/main/Lab8/Lab8_FT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers
from keras.models import load_model
from keras.applications import VGG19
from keras import layers
from keras import Model
import matplotlib.pyplot as plt

In [ ]:
dir = "/content/drive/MyDrive/Escuela/Octavo Semestre/DSDM/data"

VGG_SIZE = 224

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                zoom_range=0.1,
                                horizontal_flip=True,
                                validation_split=0.2)

In [ ]:
train_generator = datagen.flow_from_directory(dir,
                                                target_size=(VGG_SIZE, VGG_SIZE), 
                                                color_mode='rgb',
                                                batch_size=20,
                                                class_mode='sparse',
                                                subset='training')

val_generator = datagen.flow_from_directory(dir,
                                                target_size=(VGG_SIZE, VGG_SIZE), 
                                                color_mode='rgb',
                                                batch_size=20,
                                                class_mode='sparse',
                                                subset='validation')

Found 648 images belonging to 4 classes.
Found 160 images belonging to 4 classes.


In [ ]:
transfer_model = VGG19(include_top=True,
                        weights="imagenet",
                        classes=1000)
input = transfer_model.inputs

574717952/574710816 [==============================] - 5s 0us/step


In [ ]:
x = transfer_model.get_layer("block5_pool").output
x = layers.Flatten(name='flatten')(x)
x = layers.Dense(1024)(x)
x = layers.Dropout(0.1)
x = layers.Dense(1024)(x)
output_layer = layers.Dense(4, activation='softmax')(x)

model = Model(input, output_layer)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
history = model.fit(train_generator,
                      batch_size=40,
                      epochs=10, 
                      validation_data=val_generator)

Epoch 1/10
17/33 [==============>...............] - ETA: 7:27 - loss: 12.2725 - acc: 0.2712

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()